In [11]:
import pandas as pd
import numpy as np
import numpy as np
import os as os
import time

In [2]:
plans_df = pd.read_csv('plans.csv')
zips_df = pd.read_csv('zips.csv')
slcsp_df = pd.read_csv('slcsp.csv')

In [3]:
silverplans_df = plans_df[plans_df['metal_level'] == 'Silver']
silverplans_df.head(10)
#silverplans_df.shape

,plan_id,state,metal_level,rate,rate_area
0,74449NR9870320,GA,Silver,298.62,7
1,26325VH2723968,FL,Silver,421.43,60
5,36956VI7724021,TX,Silver,310.12,11
12,72190DW3284384,AZ,Silver,353.37,4
17,22521KG6880321,GA,Silver,285.76,13
18,52890CP9247848,PA,Silver,264.46,5
19,71371IL9623602,WI,Silver,348.73,11
20,82206LV3597031,MI,Silver,321.99,6
21,25775GT9423594,MT,Silver,262.93,1
22,65128VB5535475,SC,Silver,266.62,25


In [4]:
zips_df.head(10)
#zips_df.shape

,zipcode,state,county_code,name,rate_area
0,36749,AL,1001,Autauga,11
1,36703,AL,1001,Autauga,11
2,36003,AL,1001,Autauga,11
3,36008,AL,1001,Autauga,11
4,36006,AL,1001,Autauga,11
5,36022,AL,1001,Autauga,11
6,36051,AL,1001,Autauga,11
7,36091,AL,1001,Autauga,11
8,36068,AL,1001,Autauga,11
9,36067,AL,1001,Autauga,11


In [5]:
slcsp_df.head(10)

,zipcode,rate
0,64148,NaN
1,67118,NaN
2,40813,NaN
3,18229,NaN
4,51012,NaN
5,79168,NaN
6,54923,NaN
7,67651,NaN
8,49448,NaN
9,27702,NaN


In [6]:
# merging

silver_by_zip_df = zips_df.merge(silverplans_df, how="left", on=['state','rate_area'])
silver_by_zip_df.head(10)

,zipcode,state,county_code,name,rate_area,plan_id,metal_level,rate
0,36749,AL,1001,Autauga,11,03584UW8758085,Silver,268.26
1,36749,AL,1001,Autauga,11,28130ET1465358,Silver,256.21
2,36749,AL,1001,Autauga,11,14324EQ6876885,Silver,271.77
3,36749,AL,1001,Autauga,11,31507CI3685181,Silver,261.81
4,36749,AL,1001,Autauga,11,84233GD1213907,Silver,263.30
5,36749,AL,1001,Autauga,11,82065QL7561635,Silver,248.17
6,36703,AL,1001,Autauga,11,03584UW8758085,Silver,268.26
7,36703,AL,1001,Autauga,11,28130ET1465358,Silver,256.21
8,36703,AL,1001,Autauga,11,14324EQ6876885,Silver,271.77
9,36703,AL,1001,Autauga,11,31507CI3685181,Silver,261.81


In [7]:
# compile a drop list for zipcodes with multiple rate_areas 
zipcode_areas_df = pd.DataFrame(columns=['zipcode','area_count'])

zips = zips_df.zipcode.unique()


z = 36749
temp = silver_by_zip_df[silver_by_zip_df['zipcode'] == z]
rate_areas = temp['rate_area'].unique().size
rate_areas
#print(z,' has ',count,' rate_areas')


2

In [13]:
start_time = time.time()
# compile a drop list for zipcodes with multiple rate_areas 
zipcode_areas_df = pd.DataFrame(columns=['zipcode','area_count'])

zips = zips_df.zipcode.unique()

# run through it
for z in zips:
    temp = silver_by_zip_df[silver_by_zip_df['zipcode'] == z]
    count = temp['rate_area'].unique().size
    zipcode_areas_df = zipcode_areas_df.append({
        'zipcode':z,
        'area_count':count
    }, ignore_index=True)

end_time = time.time()

print('execution took', end_time - start_time)

execution took 126.66041398048401


In [12]:
start_time = time.time()
# EXPERIMENTAL - DO NOT EXCUTE
# Create a dictionary of the number of unique area codes per zipcode

# Empty dictory
zipcode_ratearea_dict = {}
# Checks if key of a is in dictionary
a = 54000
print('a is in dict : ',a in zipcode_ratearea_dict)

for index, row in silver_by_zip_df.iterrows():
    if str(row['zipcode']) in zipcode_ratearea_dict:
        zipcode_ratearea_dict[str(row['zipcode'])].append(row['rate_area'])
    else:
        zipcode_ratearea_dict[str(row['zipcode'])] = [row['rate_area']]

print('length of dict',len(zipcode_ratearea_dict))

end_time = time.time()

print('execution took', end_time - start_time)

a is in dict :  False
length of dict 38804
execution took 90.44069576263428


In [21]:
print('before drop',zipcode_areas_df.shape)
zipcode_areas_df.head(10)


zipcode_areas_df = zipcode_areas_df[zipcode_areas_df['area_count'] == 1]

print('after drop',zipcode_areas_df.shape)
zipcode_areas_df.head(10)

before drop (38804, 2)
after drop (35090, 2)


,zipcode,area_count
2,36003,1
3,36008,1
5,36022,1
8,36068,1
9,36067,1
11,36066,1
12,36562,1
13,36561,1
14,36530,1
15,36507,1


In [51]:
# go and drop the rows with zips with multiple rate_areas
# compile unique list
zips = zipcode_areas_df.zipcode.unique()

print('before: ',silver_by_zip_df.shape)
silver_by_zip_df = silver_by_zip_df[silver_by_zip_df.zipcode.isin(zips)]
print('after : ',silver_by_zip_df.shape)
silver_by_zip_df.head(10)

before:  (711207, 8)
after :  (711207, 8)


,zipcode,state,county_code,name,rate_area,plan_id,metal_level,rate
12,36003,AL,1001,Autauga,11,03584UW8758085,Silver,268.26
13,36003,AL,1001,Autauga,11,28130ET1465358,Silver,256.21
14,36003,AL,1001,Autauga,11,14324EQ6876885,Silver,271.77
15,36003,AL,1001,Autauga,11,31507CI3685181,Silver,261.81
16,36003,AL,1001,Autauga,11,84233GD1213907,Silver,263.30
17,36003,AL,1001,Autauga,11,82065QL7561635,Silver,248.17
18,36008,AL,1001,Autauga,11,03584UW8758085,Silver,268.26
19,36008,AL,1001,Autauga,11,28130ET1465358,Silver,256.21
20,36008,AL,1001,Autauga,11,14324EQ6876885,Silver,271.77
21,36008,AL,1001,Autauga,11,31507CI3685181,Silver,261.81


In [53]:
# go and drop the rows with zips with only 1 rate
zips = silver_by_zip_df.zipcode.unique()
zipcode_rates_df = pd.DataFrame(columns=['zipcode','rates_count'])
for z in zips:
    temp = silver_by_zip_df[silver_by_zip_df['zipcode'] == z]
    count = temp['rate'].unique().size
    zipcode_rates_df = zipcode_rates_df.append({
        'zipcode':z,
        'rates_count':count
    }, ignore_index=True)

print('before drop',zipcode_rates_df.shape)
zipcode_areas_df.head(10)


zipcode_rates_df = zipcode_rates_df[zipcode_rates_df['rates_count'] > 1]

print('after drop',zipcode_rates_df.shape)
zipcode_rates_df.head(10)

before drop (35090, 2)
after drop (25215, 2)


,zipcode,rates_count
0,36003,6
1,36008,6
2,36022,6
3,36068,6
4,36067,6
5,36066,6
6,36562,6
7,36561,6
8,36530,6
9,36507,6


In [54]:
# go and drop the rows with zips with multiple rate_areas
# compile unique list
zips = zipcode_rates_df.zipcode.unique()

print('before: ',silver_by_zip_df.shape)
silver_by_zip_df = silver_by_zip_df[silver_by_zip_df.zipcode.isin(zips)]
print('after : ',silver_by_zip_df.shape)
silver_by_zip_df.head(10)

before:  (711207, 8)
after :  (700153, 8)


,zipcode,state,county_code,name,rate_area,plan_id,metal_level,rate
12,36003,AL,1001,Autauga,11,03584UW8758085,Silver,268.26
13,36003,AL,1001,Autauga,11,28130ET1465358,Silver,256.21
14,36003,AL,1001,Autauga,11,14324EQ6876885,Silver,271.77
15,36003,AL,1001,Autauga,11,31507CI3685181,Silver,261.81
16,36003,AL,1001,Autauga,11,84233GD1213907,Silver,263.30
17,36003,AL,1001,Autauga,11,82065QL7561635,Silver,248.17
18,36008,AL,1001,Autauga,11,03584UW8758085,Silver,268.26
19,36008,AL,1001,Autauga,11,28130ET1465358,Silver,256.21
20,36008,AL,1001,Autauga,11,14324EQ6876885,Silver,271.77
21,36008,AL,1001,Autauga,11,31507CI3685181,Silver,261.81


In [58]:
# Now the data is cleaned and ready for analysis to find the second lowest rate
# Only Silver plans
# Zipcodes with only 1 rate area
# Zipcodes with more than one rate price

# Compile a list of prices by zip code
# Sort from least to greatest, stop after 2nd run through
# Update the other csv


zips = silver_by_zip_df.zipcode.unique()
for z in zips:
    temp = silver_by_zip_df[silver_by_zip_df['zipcode'] == z]
    vals = temp['rate'].unique()
    minimum = min(vals)
    #new_arr = np.delete(arr, np.where(arr == 2))
    vals = np.delete(vals, np.where(vals == minimum))
    secondMin = min(vals)
    #df.loc[df.ID == 103, 'FirstName'] = "Matt"
    slcsp_df.loc[slcsp_df.zipcode == z, 'rate'] = secondMin

In [59]:
# show results
slcsp_df.head(20)

,zipcode,rate
0,64148,245.20
1,67118,212.35
2,40813,NaN
3,18229,231.48
4,51012,252.76
5,79168,243.68
6,54923,NaN
7,67651,249.44
8,49448,221.63
9,27702,283.08


In [67]:
failed_zip = 48872
test_df = zips_df.loc[zips_df.zipcode == failed_zip]
test_df

,zipcode,state,county_code,name,rate_area
20779,48872,MI,26065,Ingham,7
21070,48872,MI,26093,Livingston,4
21661,48872,MI,26155,Shiawassee,5
